In [4]:
import cv2 
from math import *
import math
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import csv
import numpy
import random
from tabulate import tabulate

alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
            'y', 'z', '0', '1', '2', '3', '4', '5',
            '6', '7', '8', '9']

def toBlackWhiteBinary(img):
    
    for i in range(len(img)):
        for j in range(len(img[i])):
            if img[i][j] < 128:
                img[i][j] = 0
                
            else:
                img[i][j] = 255
            
    
    return img

def makeBinaryImages():                
    for i in alphabet:
        path = list("./CharDataset1/x.jpg")
        path[len(path) - 5] = str(i)
        path = "".join(path)
        print(path, "\n")
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        print(type(img))
        img = toBlackWhiteBinary(img)
        cv2.imwrite(path, img)
        
def ImageToArray(img):
    final = []
    for i in range(len(img)):
        for j in range(len(img[1])):
            final.append(img[i][j])
            
    for i in range(len(final)):
        if final[i] == 255:
            final[i] = 1
        else:
            final[i] = 0
    return final

def compareImage(Original, Output):
    mismatch = 0
    for i in range(len(Original)):
        if Output[i] != Original[i]:
            mismatch += 1
    return mismatch / len(Original)

#gets all dataset 1 images
def DataSetOfAllImages():
    dataset = []
    for i in alphabet:
        path = list("./CharDataset1/x.jpg")
        path[len(path) - 5] = str(i)
        path = "".join(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img.setflags(write = 1)
       
        dataset.append(ImageToArray(img))
    return dataset

def Threshold(Arr):
    newArr = []
    for i in Arr:
        if i > 0.5:
            newArr.append(1)
        else:
            newArr.append(0)
    return newArr

def ThresholdResultArrays(arrs):
    newArr = []
    for i in arrs:
        newArr.append(Threshold(i))
    return newArr

def CountWhitePixels(img):
    count = 0
    for i in img:
        if i == 0:
            count += 1
    return count

def ComputeFh(correctImage, Output):
    TotalBlackPixels = 0
    TotalCorrectPixels = 0
    
    for i in range(len(correctImage)):
        if correctImage[i] == 1:
            TotalBlackPixels += 1
            if(Output[i] == 1):
                TotalCorrectPixels += 1
    return TotalCorrectPixels / TotalBlackPixels

def ComputeFhArray(correctImages, OutputImages):
    fhArr = []
    for i in range(len(correctImages)):
        fhArr.append(ComputeFh(correctImages[i], OutputImages[i]))
    return fhArr

def ComputeFfa(correctImage, Output):
    TotalWrongPixels = 0
    
    for i in range(len(correctImage)):
        if (Output[i] == 1) and (correctImage[i] != 1):
            TotalWrongPixels += 1
    
    return TotalWrongPixels / CountWhitePixels(correctImage)

def ComputeFfaArray(correctImages, OutputImages):
    ffaArr = []
    for i in range(len(correctImages)):
        ffaArr.append(ComputeFfa(correctImages[i], OutputImages[i]))
    return ffaArr

#Images would be X
def Perturb(images, mean, std_dev):
    
    NoiseCorruptedImages = []
    
    for i in images:
        currentImage = i
        
        mean, std_dev = mean, std_dev

        sample = numpy.random.normal(mean, std_dev, 25)

        RandomImageIndexes = random.sample(range(0, 255), 25)
        
        sampleIndex = 0
        
        for j in RandomImageIndexes:
            currentImage[j] += sample[sampleIndex]
            sampleIndex += 1
        NoiseCorruptedImages.append(currentImage)
        
    return ThresholdResultArrays(NoiseCorruptedImages)
            
    
    
def Model(X, y):
    
    model = Sequential()
    model.add(Dense(256, input_dim = 256, activation='relu'))
    #model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.fit(X, y, epochs=10, batch_size=2, verbose = True)
    return model

def MultiLayerModel(X, y):
    
    model = Sequential()
    model.add(Dense(128, input_dim = 256, activation='relu'))
    model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.fit(X, y, epochs=20, batch_size=2, verbose = True)
    return model

def RunAnalysis(X, Y, model, plotTitle=None):
    
    FhArray = ComputeFhArray(X, Y)
    FfaArray = ComputeFfaArray(X, Y)
    
    
    #print(FhArray)
    #print(FfaArray)
    
    plt.scatter(FfaArray, FhArray)
    plt.title(plotTitle)
    plt.xlabel("Ffa")
    plt.ylabel("Fh")
    plt.show()
    
    return [FhArray, FfaArray]

def Tabulate(FhArrays, FfaArrays):
    #(["Image 0",FhStdDev0, FfaStdDev0, ..., FhStdDev0.1, FfaStdDev0.1])
    table = []
    for i in range(10):
        newEntry = []
        newEntry.append(str(i))
        for j in range(len(FhArrays)):
            newEntry.append(str(round(FhArrays[i][j], 2)))
            newEntry.append(str(round(FfaArrays[i][j], 2)))
        table.append(newEntry)
    print(tabulate(table, headers = ["Image", "Fh 0", "Ffa 0", "Fh 0.001", "Ffa 0.001",
                                     "Fh 0.002", "Ffa 0.002", "Fh 0.003", "Ffa 0.003", 
                                     "Fh 0.005", "Ffa 0.005", "Fh 0.01", "Ffa 0.01", 
                                    "Fh 0.02", "Ffa 0.02", "Fh 0.03", "Ffa 0.03", 
                                     "Fh 0.05", "Ffa 0.05", "Fh 0.1", "Ffa 0.1"]))
    
def FinalPlot(FhArrays, FfaArrays, Std_Devs):
    
    for i in range(len(FhArrays)):
        for j in range(len(FhArrays[i])):
            for k in Std_Devs:
                plt.scatter(k, FhArrays[i][j], color='blue')
                plt.scatter(k, FfaArrays[i][j], color = 'red')
    plt.title("Graph of Fh and Ffa vs. Noise Standard Deviation for noise-corrupted Alphanumeric Imagery (16x16 pixels) for Autoassociative Single-Layer Perceptron")
    
    plt.xscale('log')
    plt.xlabel("Gaussian Noise Level (stdev, at 10 pct xsecn)")
    plt.xlim([0, 0.1])
    #plt.xticks(numpy.arange(0, 0.1, 0.001))
    
    plt.ylim([0, 1])
    plt.ylabel("Fh and Ffa")
    
    plt.legend(["Blue = Fh", "Red = Ffa"])
    
    plt.show()
        
def RunExperiment(X, M):
    #print(X[1])
    Y = ThresholdResultArrays(M.predict(X))
    a = RunAnalysis(X, Y, M, "Fh Vs. Ffa 0 Std. Dev")
    
    
    stdDevs = [0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
    
    FhArrays = []
    FfaArrays = []
    FhArrays.append(a[0])
    FfaArrays.append(a[1])
    
    for i in stdDevs:
        NoisyImages = Perturb(X, 0, i)
        a = RunAnalysis(NoisyImages, Y, M, "Fh Vs. Ffa Noise with Std Dev = " + str(i))
        FhArrays.append(a[0])
        FfaArrays.append(a[1])
        
    Tabulate(FhArrays, FfaArrays)
    stdDevs = [0] + stdDevs
    FinalPlot(FhArrays, FfaArrays, stdDevs)
    
def main():
            
    X = numpy.array(DataSetOfAllImages())
    y = X
    
    #M = Model(X, y)
    #RunExperiment(X, M)
    
    print("Testing Multi Layer Perceptron")
    
    M = MultiLayerModel(X, y)
    RunExperiment(X, M)
        
        

        
        



makeBinaryImages()






./CharDataset1/a.jpg 

<class 'numpy.ndarray'>
./CharDataset1/b.jpg 

<class 'numpy.ndarray'>
./CharDataset1/c.jpg 

<class 'numpy.ndarray'>
./CharDataset1/d.jpg 

<class 'numpy.ndarray'>
./CharDataset1/e.jpg 

<class 'numpy.ndarray'>
./CharDataset1/f.jpg 

<class 'numpy.ndarray'>
./CharDataset1/g.jpg 

<class 'numpy.ndarray'>
./CharDataset1/h.jpg 

<class 'numpy.ndarray'>
./CharDataset1/i.jpg 

<class 'numpy.ndarray'>
./CharDataset1/j.jpg 

<class 'numpy.ndarray'>
./CharDataset1/k.jpg 

<class 'numpy.ndarray'>
./CharDataset1/l.jpg 

<class 'numpy.ndarray'>
./CharDataset1/m.jpg 

<class 'numpy.ndarray'>
./CharDataset1/n.jpg 

<class 'numpy.ndarray'>
./CharDataset1/o.jpg 

<class 'numpy.ndarray'>
./CharDataset1/p.jpg 

<class 'numpy.ndarray'>
./CharDataset1/q.jpg 

<class 'numpy.ndarray'>
./CharDataset1/r.jpg 

<class 'numpy.ndarray'>
./CharDataset1/s.jpg 

<class 'numpy.ndarray'>
./CharDataset1/t.jpg 

<class 'numpy.ndarray'>
./CharDataset1/u.jpg 

<class 'numpy.ndarray'>
./CharDataset